# BrainBuilder Column
This notebook builds a microcircuit using the brainbuilder library and some custom-built atlases to match the existing BBP model.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from brainbuilder.sscx.annotation import build_column_atlas
from brainbuilder.utils import bbp

from voxcell.core import CellCollection, VoxelData, Hierarchy

from voxcellview.widget import VoxcellWidget

## Atlas and Density

In [ ]:
# data from the recipe:
recipe_filename = "../brainbuilder/data/bbp_recipe/builderRecipeAllPathways.xml"
neurondb_filename = "../brainbuilder/data/bbp_recipe/neurondb.dat"

In [ ]:
annotation, hierarchy = build_column_atlas(recipe_filename)
region_map = { x: str(x,) for x in range(1, 7) }

In [ ]:
VoxcellWidget().show_volume('annotation', annotation.raw)

Build column density using the layer percentages from the recipe and our custom annotation

In [ ]:
column_density = bbp.load_recipe_density(recipe_filename, annotation, region_map)

## Circuit Building

In [ ]:
column_cell_count = bbp.get_total_neurons(recipe_filename)

In [ ]:
from voxcell.positions import create_cell_positions

In [ ]:
recipe_sdist = bbp.load_recipe_as_spatial_distribution(recipe_filename, annotation, region_map)
neuron_sdist = bbp.load_neurondb_v4_as_spatial_distribution(neurondb_filename, annotation, region_map, percentile=0.92)

def build_cells(total_cell_count, density):
    '''the standard workflow to build an SSCX circuit'''
    cells = CellCollection()

    # Positions
    cells.positions = create_cell_positions(density, total_cell_count)
    
    # Layer
    cells.add_properties(
        pd.DataFrame({'layer': annotation.lookup(cells.positions).astype(np.str)})
    )
    
    # MType / EType / Synapse Class / mClass
    cells.add_properties(
        recipe_sdist.collect(
            positions=cells.positions,
            preassigned=cells.properties[['layer']],
            names=['mtype', 'etype', 'synapse_class', 'mClass']
        )
    )

    # Morphology
    cells.add_properties(
        neuron_sdist.collect(
            positions=cells.positions,
            preassigned=cells.properties[['layer', 'mtype', 'etype']],
            names=['morphology']
        )
    )

    return cells

In [ ]:
single_columns = [build_cells(column_cell_count, column_density) for i in range(7)]

In [ ]:
VoxcellWidget().show_property('mtype', single_columns[0].as_dataframe())

## Analysis

In [ ]:
import report

In [ ]:
report.report_height_histogram(single_columns)

In [ ]:
recipe = bbp.get_distribution_from_recipe(recipe_filename)

Compare the percentages of cells grouped by layer with the values specified in the recipe.

In [ ]:
report.report_cell_percentages(recipe, [single_columns[0]], ['layer'])

Compare the percentages of cells grouped by layer-mtype-etype with the values specified in the recipe.

In [ ]:
report.report_cell_percentages(recipe, [single_columns[0]], ['layer', 'mtype', 'etype'])

## Placement hints
Look at cells without valid morphologies, check that they belong to mtypes for which some placement hint scores are zero

In [ ]:
neurondb = bbp.load_neurondb_v4(neurondb_filename)

In [ ]:
hints = report.collect_placement_hints(neurondb)
for c in single_columns:
    report.check_bad_bins(hints, c)